# Evaluate detector and classifier together

In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras import layers
from matplotlib import pyplot as plt
import tensorflow as tf

In [ ]:
# Detect hardware
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
from keras.layers import MultiHeadAttention, Input, Dense, Softmax, Conv1D, Dropout, GlobalAveragePooling1D
from keras.layers import LayerNormalization, Layer
from keras.layers import TextVectorization, Embedding
from tensorflow import convert_to_tensor
from keras import Model, Sequential
import math
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [5]:
keras.saving.get_custom_objects().clear()
@keras.saving.register_keras_serializable(package="MyLayers")
class EncoderLayer(tf.keras.Model):
    def __init__(self, total_heads, dense_units, sequence_length, embed_dim, classes, **kwargs):
        super(EncoderLayer, self).__init__(**kwargs)
        self.total_heads = total_heads
        self.dense_units = dense_units
        self.sequence_length = sequence_length
        self.embed_dim = embed_dim
        self.classes = classes
        self.multihead = MultiHeadAttention(num_heads=total_heads, key_dim=embed_dim)
        self.forward = Sequential([Dense(self.embed_dim)])
        self.normalize_layer = LayerNormalization()

    def call(self, x):
        multihead_layer = self.multihead(x,x)
        normalize1_layer = self.normalize_layer(x + multihead_layer)
        forward_layer = self.forward(normalize1_layer)
        normalize2_layer = self.normalize_layer(normalize1_layer + forward_layer)
        final_output = self.forward(normalize2_layer)
        return final_output

    def get_config(self):
        base_config = super().get_config()
        return {"total_heads": self.total_heads,"dense_units": self.dense_units,
                "sequence_length": self.sequence_length,"embed_dim": self.embed_dim,
                "classes": self.classes, **base_config}
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
from keras.models import load_model
import tensorflow as tf
detector = keras.models.load_model("/content/drive/MyDrive/Colab/models/model_conv1d.keras", compile=False)
detector.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mse')
detector.summary()

In [ ]:
from keras.models import load_model
import tensorflow as tf
classifier = keras.models.load_model("/content/drive/MyDrive/Colab/models/transformer.keras", compile=False)
classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mse')
classifier.summary()

In [ ]:
# leer datos normales
pd.options.mode.copy_on_write = True #to save on memory and avoid making copies of data until written to
my_file = '/content/drive/MyDrive/Colab/UdeA/normalized/edge1/edge1_11.csv'
df = pd.read_csv(my_file)
df = df.reindex(['localtime', 'indoor_temp', 'indoor_humidity', 'edge_soc_temp', 'edge_load1', 'edge_load15',
      'delta_sent', 'delta_recv', 'edge_current', 'motor_sound', 'motor_freq', 'motor_volt',
      'motor_current', 'motor_recipe','edge_cpu_freq', 'edge_load5','edge_cpu_voltage', 'edge_memory_free',
      'edge_memory_avail', 'fault_type', 'attack_type', 'anomaly', 'edge_wifi_send', 'edge_wifi_receiv',
      'normal', 'E', 'A', 'F', 'N'], axis=1)
df = df.iloc[:,:]
x_test=df.iloc[:,3:11]
print(x_test.shape)

In [9]:
#Every time a new autoencoder is trained, new threshold values ​​must be calculated for each variable.
threshold = np.array([0.343109, 0.275433, 0.393678, 0.573704, 0.220106, 0.396813, 0.2229767, 0.638695])

In [14]:
# Predicting and Classifing Anomalies
anomaly = ['A', 'F', 'E']
N_TIMES = 300
pred_detect = []
classes = []
with tf.device('/device:GPU:0'):
  i = N_TIMES
  while i < (x_test.shape[0] - 1):
    df_original = np.expand_dims(x_test[i-N_TIMES:i], axis=0)
    diff = np.abs(detector.predict([df_original], verbose=False) - df_original)
    diff = diff.reshape(diff.shape[1], diff.shape[2])
    pred_detect += list(np.sum(diff > threshold, axis=1)/(np.sum(diff > threshold, axis=1)+0.0001))

    if np.sum(pred_detect[i-N_TIMES:i]) > 15:
      predicted = classifier.predict([df_original], verbose=False)
      predicted = predicted.reshape(predicted.shape[1], predicted.shape[2])
      classes += list([anomaly[np.argmax(predicted, axis=1)[j]] for j in range(N_TIMES)])
    else:
      classes += list(np.full(N_TIMES, 'N'))
    i+=N_TIMES

In [ ]:
from sklearn.metrics import classification_report
true_labels = list(df.iloc[:len(classes),22])
predicted_labels = classes
print(classification_report(true_labels, predicted_labels))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
# Create a confusion matrix
confusion_matrix = confusion_matrix(true_labels, predicted_labels, labels=['A','E','F','N'])
print(confusion_matrix)
# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix, display_labels=['A','E','F','N'])
disp.plot()

In [ ]:
#Plotting the most important results
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

fig = make_subplots(
    rows=4, cols=1,
    vertical_spacing=0.15,
    subplot_titles=("Detection","Classification", "Attack", "Failure")
    )
fig.add_trace(
    go.Scatter( y=pred_detect[:]),
    row=1, col=1
)
fig.add_trace(
    go.Scatter( y=classes[:]),
    row=2, col=1
)
fig.add_trace(
    go.Scatter( y=df.iloc[:,21]),
    row=3, col=1
)
fig.add_trace(
    go.Scatter( y=df.iloc[:,20]),
    row=4, col=1
)

fig.update_layout(height=600, width=900, showlegend=False,
                  title_text="Anomaly Classification in IIoT Systems", title_x=0.5)
fig.show()
